In [ ]:
from tqdm.auto import tqdm
import multiprocessing
import sys
from datetime import datetime
from time import sleep
import datetime as dt
import pandas as pd

from Pandora.helper import TDays
from Pandora.research.backtest import CODES_SHORT, calc_sharpe, COMMISSION, CODES_TRADABLE_SL
from vnpy.app.vnpy_portfoliostrategy.strategies.macd_bband_strategy import MACDBBANDStrategy, MACDBBANDX2DStrategy
from vnpy.trader.constant import Interval, Product
from vnpy.trader.optimize import OptimizationSetting
from vnpy.trader.setting import SETTINGS
from logging import INFO
from vnpy.app.vnpy_portfoliostrategy import BacktestingEngine

SETTINGS["log.active"] = True
SETTINGS["log.level"] = INFO
SETTINGS["log.console"] = True



In [ ]:
def get_mc(codes):
    from vnpy.trader.database import get_database

    database = get_database()
    today = datetime.combine(TDays.get_tday(fmt=None), dt.time(0))
    contracts = database.load_contract_data(product=Product.FUTURES, start=today, end=today)
    mc = [i for i in contracts if i.symbol.endswith('00') and i.product_id in codes]

    return mc


In [ ]:
mc = get_mc(CODES_SHORT)
# mc = get_mc(CODES_SHORT | {'lc', 'ec', 'si', 'ao'})


In [ ]:
engine = BacktestingEngine()
engine.contracts = {i.vt_symbol: i for i in mc}

engine.set_parameters(
    vt_symbols=[i.vt_symbol for i in mc],
    interval=Interval.MINUTE_15,
    start=datetime(2015, 1, 1),
    end=datetime(2024, 8, 16),
    rates={
        i.vt_symbol: COMMISSION for i in mc
    },
    slippages={
        i.vt_symbol: 0 for i in mc
    },
    sizes={
        i.vt_symbol: i.size for i in mc
    },
    priceticks={
        i.vt_symbol: i.pricetick for i in mc
    },
    capital=1e8,
)

engine.load_data()



In [ ]:
engine.clear_data()
setting = {
    "window": 350,
    "bband_width": 1.5,
}
engine.add_strategy(MACDBBANDStrategy, setting)

engine.run_backtesting()
df = engine.calculate_result()

daily_base = pd.Series(df['net_pnl'] / engine.capital, index=pd.to_datetime(df.index))
daily_base.cumsum().plot(figsize=(16, 9))
calc_sharpe(daily_base)

In [ ]:
desc, rounds = engine.describe_trades()
rounds

In [ ]:
from vnpy.trader.constant import Direction

def rounds_to_open_signal(rounds):
    open_signal = pd.DataFrame()
    
    for i in range(len(rounds)):
        symbol = rounds['symbol'].iat[i]
        direction = rounds['direction'].iat[i]
        open_time = rounds['open_time'].iat[i]
        close_time = rounds['close_time'].iat[i]
        
        if direction == Direction.LONG:
            open_signal.loc[open_time, symbol] = 1
        else:
            open_signal.loc[open_time, symbol] = -1
    
        open_signal.loc[close_time, symbol] = 0
    
    open_signal.index = open_signal.index.tz_localize(None)
    open_signal = open_signal.sort_index()
    
    return open_signal

In [ ]:
from Pandora.constant import Frequency
from Pandora.research import *

codes = CODES_TRADABLE_SL
quote_bt, ret = get_quote(codes, end=dt.datetime.now(), freq=Frequency.Min_15)
weight = get_weight_by_std_minus(quote_bt, 100, n=3)


In [ ]:
weight = weight.loc[:datetime(2024, 8, 16), :]
ret = ret.loc[:datetime(2024, 8, 16), :]
open_signal = rounds_to_open_signal(rounds)
_, open_signal = ret.align(open_signal, join='left')
daily_base = backtest_factor(open_signal, weight, ret, COMMISSION)

daily_base.cumsum().plot(figsize=(16, 9))
calc_sharpe(daily_base)

## Full set

- CODES_SHORT
- Vol exit
- LS imba

### ATR multiplier = 11

In [ ]:
setting = {
    "window": 50,
}
engine.add_strategy(MACDBBANDX2DStrategy, setting)


setting = OptimizationSetting()
setting.set_target("sharpe_ratio")
setting.add_parameter("window", 50, 500, 50)
setting.add_parameter("bband_width", 1.5, 2, 0.25)
setting.add_parameter("atr_multiplier", 11)
setting.add_parameter("weighting_method", 0, 3, 1)

opt_res = engine.run_bf_optimization(setting, max_workers=10)

In [ ]:
for res in opt_res:
    rounds = res[3]
    open_signal = rounds_to_open_signal(rounds)
    
    params = '_'.join((str(k) + "_" + str(v) for k, v in res[0].items()))
    strategy_id = f"MACDBBANDStrategy_{params}"
    open_signal.to_parquet(strategy_id + ".parquet")

In [ ]:
sharpe_mat = []
for res in opt_res:
    
    param = res[0].copy()
    sharpe = res[1]
    stat = res[2]
    
    param.update(stat)
    sharpe_mat.append(param)
    
stats = pd.DataFrame(sharpe_mat)
stats

In [ ]:

for bband_width in [1.5, 1.75, 2]:
    loc = (stats['bband_width'] == bband_width)
    tmp = stats[loc]
    mat = tmp.pivot(index='window', columns='weighting_method', values='sharpe_ratio')
    
    display(f"bband_width = {bband_width}")
    display(mat)
    display(mat.mean())

In [ ]:
sharpe_mat = pd.DataFrame()

for res in opt_res:
    param = res[0]
    sharpe = res[1]
    
    sharpe_mat.loc[param['window'], param['bband_width']] = sharpe
    
sharpe_mat.sort_index().iloc[:, [0, 2, 1]]


### ATR multiplier = 13

In [ ]:
setting = {
    "window": 50,
}
engine.add_strategy(MACDBBANDX2DStrategy, setting)


setting = OptimizationSetting()
setting.set_target("sharpe_ratio")
setting.add_parameter("window", 50, 500, 50)
setting.add_parameter("bband_width", 1.5, 2, 0.25)
setting.add_parameter("atr_multiplier", 13)
setting.add_parameter("weighting_method", 0, 3, 1)

opt_res = engine.run_bf_optimization(setting, max_workers=30)

In [ ]:
sharpe_mat = []
for res in opt_res:
    
    param = res[0].copy()
    sharpe = res[1]
    stat = res[2]
    
    param.update(stat)
    sharpe_mat.append(param)
    
stats = pd.DataFrame(sharpe_mat)

for bband_width in [1.5, 1.75, 2]:
    loc = (stats['bband_width'] == bband_width)
    tmp = stats[loc]
    mat = tmp.pivot(index='window', columns='weighting_method', values='sharpe_ratio')
    
    display(f"bband_width = {bband_width}")
    display(mat)
    display(mat.mean())

In [ ]:
setting = {
    "window": 50,
}
engine.add_strategy(MACDBBANDStrategy, setting)


setting = OptimizationSetting()
setting.set_target("sharpe_ratio")
setting.add_parameter("window", 50, 500, 50)
setting.add_parameter("bband_width", 1.5, 2, 0.25)
setting.add_parameter("atr_multiplier", 13)

opt_res = engine.run_bf_optimization(setting, max_workers=5)

sharpe_mat = pd.DataFrame()

for res in opt_res:
    param = res[0]
    sharpe = res[1]

    sharpe_mat.loc[param['window'], param['bband_width']] = sharpe

sharpe_mat.sort_index()

In [ ]:
for res in opt_res:
    rounds = res[3]
    open_signal = rounds_to_open_signal(rounds)
    
    params = '_'.join((str(k) + "_" + str(v) for k, v in res[0].items()))
    strategy_id = f"MACDBBANDStrategy_{params}"
    open_signal.to_parquet(strategy_id + ".parquet")

## LS balance

In [ ]:
setting = {
    "window": 50,
}
engine.add_strategy(MACDBBANDStrategy, setting)


setting = OptimizationSetting()
setting.set_target("sharpe_ratio")
setting.add_parameter("window", 50, 500, 50)
setting.add_parameter("bband_width", 1.5, 2, 0.25)
setting.add_parameter("atr_multiplier", 13)
setting.add_parameter("ls_imba", 1)

opt_res = engine.run_bf_optimization(setting, max_workers=10)

sharpe_mat = pd.DataFrame()

for res in opt_res:
    param = res[0]
    sharpe = res[1]

    sharpe_mat.loc[param['window'], param['bband_width']] = sharpe

sharpe_mat.sort_index()

## CODES_MM

In [ ]:
from Pandora.research import CODES_MM

mc = get_mc(CODES_MM)


In [ ]:
engine = BacktestingEngine()
engine.contracts = {i.vt_symbol: i for i in mc}

engine.set_parameters(
    vt_symbols=[i.vt_symbol for i in mc],
    interval=Interval.MINUTE_15,
    start=datetime(2015, 1, 1),
    end=datetime(2024, 6, 27),
    rates={
        i.vt_symbol: COMMISSION for i in mc
    },
    slippages={
        i.vt_symbol: 0 for i in mc
    },
    sizes={
        i.vt_symbol: i.size for i in mc
    },
    priceticks={
        i.vt_symbol: i.pricetick for i in mc
    },
    capital=1e8,
)

engine.load_data()



In [ ]:
setting = {
    "window": 50,
}
engine.add_strategy(MACDBBANDStrategy, setting)


setting = OptimizationSetting()
setting.set_target("sharpe_ratio")
setting.add_parameter("window", 50, 500, 50)
setting.add_parameter("bband_width", 1.5, 2, 0.25)
setting.add_parameter("atr_multiplier", 11)

opt_res = engine.run_bf_optimization(setting, max_workers=8)

In [ ]:
sharpe_mat = pd.DataFrame()

for res in opt_res:
    param = res[0]
    sharpe = res[1]
    
    sharpe_mat.loc[param['window'], param['bband_width']] = sharpe
    
sharpe_mat
